In [11]:
import cv2
import numpy as np
import winsound
import os
from ultralytics import YOLO
from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill

In [ ]:
# Cargar el modelo YOLOv8 entrenado
model = YOLO(r'Ruta al modelo\yolov8n.pt')

# Abrir cámaras
cap0 = cv2.VideoCapture(0, cv2.CAP_DSHOW)
#cap1 = cv2.VideoCapture(1, cv2.CAP_DSHOW)
#cap2 = cv2.VideoCapture(2, cv2.CAP_DSHOW)


# Ruta al archivo Excel
excel_path = r'ruta de guardado de planilla.xlsx'

# Diccionario para controlar el tiempo de la última detección por persona
ultima_deteccion = {}

# Creacion de la planilla de registro
def registrar_deteccion(persona, confianza, camara):
    ahora = datetime.now()
    hora_formateada = ahora.strftime("%d-%m-%Y %H:%M:%S")
    confianza_pct = round(confianza * 100, 2)
    clave = f"{persona}_{camara}"
    if clave in ultima_deteccion:
        tiempo_transcurrido = (ahora - ultima_deteccion[clave]).total_seconds()
        if tiempo_transcurrido < 10:
            return
    ultima_deteccion[clave] = ahora
    nueva_fila = [persona, confianza_pct, camara, hora_formateada]

    if not os.path.exists(excel_path):
        wb = Workbook()
        ws = wb.active
        ws.append(["Persona", "Confianza (%)", "Cámara", "Hora"])
        wb.save(excel_path)

    try:
        wb = load_workbook(excel_path)
        ws = wb.active
        ws.append(nueva_fila)
        wb.save(excel_path)

        # --- FORMATO CONDICIONAL ---
        last_row = ws.max_row
        cell = ws[f'B{last_row}']  # Columna B es "Confianza (%)"
        if confianza_pct > 70:
            cell.fill = PatternFill(start_color="90EE90", end_color="90EE90", fill_type="solid")  # Verde claro
        elif 50 < confianza_pct <= 70:
            cell.fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")  # Amarillo claro
        else:
            cell.fill = PatternFill(start_color="FF7F7F", end_color="FF7F7F", fill_type="solid")  # Rojo claro

        wb.save(excel_path)
        wb.close()
        print(f"[EXCEL] Registrado: {nueva_fila}")
    except Exception as e:
        print(f"[ERROR Excel] {e}")

# Procesar cada frame con detección YOLO
def process_frame(frame, camara):
    frame = cv2.resize(frame, (640, 640))
    results = model(frame)[0]

    for result in results.boxes:
        cls = int(result.cls[0])
        conf = float(result.conf[0])
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        class_name = model.names[cls]

        # Elegir color según la confianza
        if conf > 0.7:
            color = (0, 255, 0)
        elif 0.5 < conf <= 0.7:
            color = (0, 255, 255)
        else:
            color = (0, 0, 255)

        cv2.putText(frame, f'{class_name} {conf:.2f}', (x1, y1 - 10), 1, 1.3, color, 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Registrar si cumple el umbral
        if conf > 0.7:
            print(f"[DETECCIÓN] {class_name} detectado con {conf:.2f} en {camara}")
            cv2.imshow("Alerta: Residente Detectada", frame)
            registrar_deteccion(class_name, conf, camara)
        elif 0.5 < conf <= 0.7:
            print(f"[DETECCIÓN] {class_name} detectado con {conf:.2f} - Alerta en {camara}")
            cv2.imshow("Alerta: Persona Detectada", frame)
            registrar_deteccion(class_name, conf, camara)
        else:
            print(f"[DETECCIÓN] {class_name} detectado con {conf:.2f} - Ignorado en {camara}")
            cv2.imshow("Alerta: Desconocido en puerta", frame)
            registrar_deteccion("Desconocido Detectado", conf, camara)

    return frame

# Bucle principal de detección
while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret0 or not ret1 or not ret2:
        print("Error leyendo alguna cámara")
        break

    frame0 = process_frame(frame0, "Camara 0")
    frame1 = process_frame(frame1, "Camara 1")
    frame2 = process_frame(frame2, "Camara 2")

    frame0 = cv2.resize(frame0, (426, 240))
    frame1 = cv2.resize(frame1, (426, 240))
    frame2 = cv2.resize(frame2, (426, 240))

    top_row = np.hstack([frame0, frame1, frame2])
    cv2.imshow("Reconocimiento Facial - SOLO YOLOv8", top_row)

    if cv2.waitKey(1) == 27:
        break

cap0.release()
cap1.release()
cap2.release()
cv2.destroyAllWindows()